# Modelling phase

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [22]:
filepath = r'C:\Users\Asus\Desktop\DS Purwa\Module_3\Capstone Module 3\E-Commerce Churn Project\E-Commerce_Churn_Project.csv'
df_raw = pd.read_csv(filepath, index_col=0)
df = df_raw.copy()
df.head()

,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount,Churn
0,15.0,29.0,4,Laptop & Accessory,3,Single,2,0,7.0,143.32,No Churn
1,7.0,25.0,4,Mobile,1,Married,2,0,7.0,129.29,No Churn
2,27.0,13.0,3,Laptop & Accessory,1,Married,5,0,7.0,168.54,No Churn
4,30.0,15.0,4,Others,4,Single,8,0,8.0,322.17,No Churn
5,7.0,16.0,4,Mobile Phone,2,Divorced,2,0,11.0,152.81,No Churn


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2794 entries, 0 to 3940
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Tenure                    2794 non-null   float64
 1   WarehouseToHome           2794 non-null   float64
 2   NumberOfDeviceRegistered  2794 non-null   int64  
 3   PreferedOrderCat          2794 non-null   object 
 4   SatisfactionScore         2794 non-null   int64  
 5   MaritalStatus             2794 non-null   object 
 6   NumberOfAddress           2794 non-null   int64  
 7   Complain                  2794 non-null   int64  
 8   DaySinceLastOrder         2794 non-null   float64
 9   CashbackAmount            2794 non-null   float64
 10  Churn                     2794 non-null   object 
dtypes: float64(4), int64(4), object(3)
memory usage: 261.9+ KB


In [24]:
# Import necessary libraries

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Seperate the features and target variable
X = df.drop(columns='Churn')
y = df['Churn']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42,
                                                    stratify=y)


In [25]:
numerical_cols = df.select_dtypes(include=['int64', 'float64'])
categorical_cols = ['MaritalStatus', 'PreferedOrderCat']

In [26]:
# Apply column transformer to preprocess the data

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [28]:
# Define the models and sampling methods to be used in the experiment.

models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest Classifier': RandomForestClassifier(),
    'XGBoost Classifier': XGBClassifier()
}

sampling_methods = {
    'No Sampling': None,
    'SMOTE': SMOTE(),
    'Undersampling': RandomUnderSampler(),
    'Oversampling': RandomOverSampler()
}

In [29]:
result = []

In [30]:
import pandas as pd
from sklearn.metrics import (recall_score, f1_score, precision_score, accuracy_score)
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest Classifier': RandomForestClassifier(),
    'XGBoost Classifier': XGBClassifier()
}

# Define sampling methods
sampling_methods = {
    'No Sampling': None,
    'SMOTE': SMOTE(),
    'Undersampling': RandomUnderSampler(),
    'Oversampling': RandomOverSampler()
}

result = []

# Define metrics
metrics = {
    'balanced_accuracy': 'balanced_accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

# Loop through models
for model_name, model in models.items():
    skfold = StratifiedKFold(n_splits=5)
    
    # Loop through sampling methods
    for sampling_name, sampler in sampling_methods.items():
        if sampler is not None:
            # Apply the sampling method here (e.g., fit and transform)
            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
            estimator = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('models', model)])
            model_cv = cross_validate(estimator, X_resampled, y_resampled, 
                                      cv=skfold, scoring=metrics)
        else:
            estimator = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('models', model)])
            model_cv = cross_validate(estimator, X_train, y_train, 
                                      cv=skfold, scoring=metrics)
        
        # Calculate metrics
        accuracy = model_cv['test_balanced_accuracy'].mean()
        precision = model_cv['test_precision'].mean()
        recall = model_cv['test_recall'].mean()
        f1 = model_cv['test_f1'].mean()

        # Append the results to the results dataframe
        result.append({
            'Model': model_name,
            'Sampling Method': sampling_name,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1': f1
        })

# Check the result list before creating DataFrame
print("Results collected:", result)

# Create DataFrame and check structure
benchmark_results = pd.DataFrame(result)

# Check the DataFrame structure
print("Benchmark Results DataFrame:")
print(benchmark_results.head())  # Display the first few rows

# Sort results
benchmark_results.sort_values(by='Recall', ascending=False, inplace=True)

# Display the sorted results
print("Sorted Benchmark Results:")
print(benchmark_results)

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 406, in _fit
    X, fitted_transformer = fit